<a href="https://colab.research.google.com/github/google/differential-privacy/blob/main/common_docs/partition_selection_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Copyright 2021 Google LLC.
### SPDX-License-Identifier: Apache-2.0


# Optimal Partition Selection

This notebook presents Python code to enable exploration of the optimal
partition selection algorithm presented in https://arxiv.org/pdf/2006.03684.pdf.
This experimental code is only provided for exploration purposes, and should not
be used for production use cases.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(15,8)})

In [ ]:
# PreAggPartitionSelection implements optimal partition selection - instead
# of thresholding a noised value to determine whether or not a partition
# should be kept, this uses a formula derived from the
# original probablistic definition of differential privacy to generate the
# probability with which a partition should be kept. The math is shown in
# https://arxiv.org/pdf/2006.03684.pdf.
class PreaggPartitionSelection(object):

  def __init__(self, epsilon: float, delta: float):
    self._epsilon = epsilon
    self._delta = delta
    self._crossover_1 = (
        1 +
        np.floor(np.log1p(np.tanh(epsilon / 2) * (1 / delta - 1)) / epsilon))
    self._p_crossover = (np.expm1(self._crossover_1 * self.epsilon) /
                         np.expm1(self.epsilon)) * self.delta
    self._crossover_2 = self._crossover_1 + np.floor((1.0 / epsilon) * np.log1p(
        (np.expm1(epsilon) / delta) * (1 - self._p_crossover)))

  @property
  def epsilon(self):
    return self._epsilon

  @property
  def delta(self):
    return self._delta

  @property
  def first_crossover(self):
    return self._crossover_1

  @property
  def second_crossover(self):
    return self._crossover_2

  def sample_keep(self, num_users: int):
    """Sample a Bernoulli RV whether or not the variable should be exposed."""
    return np.random.Bernoulli(self.probability_of_keep(num_users))

  # ProbabilityOfKeep returns the probability with which a partition with n
  # users should be kept, Thm. 1 of https://arxiv.org/pdf/2006.03684.pdf
  def probability_of_keep(self, n: int) -> float:
   conds =  np.array([n == 0, n <= self.first_crossover, n <= self.second_crossover, 
      n > self.first_crossover])
   choicelist = np.array([0.0, ((np.expm1(n * self.epsilon) / np.expm1(self.epsilon)) *
              self.delta),
          self._p_crossover -
          (1 - self._p_crossover +
           (self.delta / np.expm1(self.epsilon))) * np.expm1(-(n - self.first_crossover) * self.epsilon),
     1.0])
   return np.select(conds, choicelist)

## Plot probabilities for varying values of $\varepsilon$ and $\delta$
The first plot shows the probability of keeping a partion as a function
of the number of users contributiong to the partition. The second plot
shows the change in probability of keeping the partition compared to a
partition with one less user. The third plot is the same, but on a
log-scale. This makes it apparent that the incremental probablities
scale exponentionally, except near the crossover points.

In [ ]:
#@title Probabilities and diffs{ run: "auto" }
epsilon = 1.034  #@param { type: "slider", min: 0.01, max: 3 , step: 0.001}
log10_delta = -7.806  #@param { type: "slider", min: -15, max: 0 , step: 0.001}

pp = PreaggPartitionSelection(epsilon, np.power(10, log10_delta))
fig, axes = plt.subplots(3, 1, figsize=(25, 15))
sns.lineplot(
    ax=axes[0],
    x=np.arange(pp.second_crossover + 2),
    y=pp.probability_of_keep(np.arange(pp.second_crossover + 2)),ci=None)
axes[0].set(ylabel='Probability of keeping partition')
axes[0].set(xlabel='Number of users')
sns.barplot(
    ax=axes[1],
    x=np.arange(pp.second_crossover + 1) + 1,
    y=np.diff(pp.probability_of_keep(np.arange(pp.second_crossover + 2))),ci=None)
axes[1].set(ylabel='Incremental probability of keeping partition')
axes[1].set(xlabel='Number of users')
sns.lineplot(
    ax=axes[2],
    x=np.arange(pp.second_crossover + 1) + 1,
    y=np.diff(pp.probability_of_keep(np.arange(pp.second_crossover + 2))),ci=None)
axes[2].set_yscale('log')
axes[2].set(ylabel='Incremental probability of keeping partition')
axes[2].set(xlabel='Number of users')
plt.plot()